# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train_data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test_data.txt'
VOCAB_SIZE = 2500

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-feature.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test_target.txt'


# Read and Load Features from .txt Files into numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_test_data[-5:]

array([[5793, 2349,    0,    2],
       [5793, 2362,    0,    1],
       [5793, 2370,    0,    4],
       [5793, 2389,    0,    3],
       [5793, 2467,    0,    1]])

In [7]:
print('Number of rows in training file: ', sparse_train_data.shape[0])
print('Number of rows in testing file: ', sparse_test_data.shape[0])

Number of rows in training file:  258374
Number of rows in testing file:  117776


In [8]:
print('Number of emails in training file: ', np.unique(sparse_train_data[:, 0]).size)

Number of emails in training file:  4013


In [9]:
print('Number of emails in testing file: ', np.unique(sparse_test_data[:, 0]).size)

Number of emails in testing file:  1724


### How to Create an empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[0:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
sparse_train_data[10][0]

0

# Create a Full Matrix from a Sparse Matrix

In [16]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Returns a dataframe.
    Arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens
    doc_idx -- position of the document id in the sparse matrix. Default: 1nd column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, non-spam is 0). Default: 3rd column
    freq_idx -- position of occurence of word in sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:, 0]) 
    full_matrix = pd.DataFrame(index=index_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
        
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [17]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 42.8 s


In [18]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculating the Probability of Spam

In [19]:
# Store the calculated value in variable called prob_spam

full_train_data.CATEGORY.size

4013

In [20]:
full_train_data.CATEGORY.sum()

1248

In [21]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is: ', prob_spam)

Probability of spam is:  0.310989284824321


## Total Number of Words/Tokens

In [22]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
email_length = full_train_features.sum(axis=1) # Count the total number of word per email
email_length.shape

(4013,)

In [24]:
email_length[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [25]:
total_wc = email_length.sum()
total_wc

429225

## Number of Tokens per Spam & Ham Emails

In [26]:
spam_lengths = email_length[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [27]:
spam_wc = spam_lengths.sum()
spam_wc

176322

In [28]:
ham_lengths = email_length[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [29]:
email_length.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0] # Testing our work

0

In [30]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252903

In [31]:
spam_wc + nonspam_wc - total_wc

0

In [32]:
print('Average number of words in spam emails: {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Average number of words in ham emails: {:.0f}'.format(nonspam_wc/ham_lengths.shape[0]))

Average number of words in spam emails: 141
Average number of words in ham emails: 91


 ## Summing the Tokens Occuring in Spam

In [33]:
full_train_features.shape

(4013, 2500)

In [34]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
train_spam_tokens.shape

(1248, 2500)

In [37]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [38]:
summed_spam_tokens.tail()

2495     2
2496    23
2497     1
2498     2
2499     8
dtype: int64

## Summing the Tokens Occuring in Ham

In [39]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [40]:
summed_ham_tokens.shape

(2500,)

In [41]:
summed_ham_tokens.tail()

2495    32
2496     9
2497    33
2498    32
2499    25
dtype: int64

In [42]:
train_ham_tokens[2499].sum() + 1

25

## P(Token | Spam) Probability that a Token Occurs given the Eamil is Spam

In [43]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012185
1    0.005229
2    0.006806
3    0.011307
4    0.006817
dtype: float64

In [44]:
prob_tokens_spam.sum()

1.0

## P(Token | Spam) - Probability that a Token Occurs given the Email is Nonspam

In [45]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + 2500)
prob_tokens_nonspam.sum()

1.0

## P(Token | Spam) - Probabiliey that a Token Occurs

In [46]:
prob_tokens_all = full_train_features.sum(axis=0) / (total_wc)
prob_tokens_all.sum()

0.9999999999999999

# Save the Trained Model

In [47]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

### Create a full Matrix from the sparse_test_data

In [48]:
sparse_test_data.shape

(117776, 4)

In [49]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 5min 46s


In [50]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [51]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)